<a href="https://colab.research.google.com/github/JorgePere27/Credit_scoring/blob/main/Credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis del riesgo de incumplimiento de los prestatarios

Este proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

Como primer paso importaremos el archivo cos los datos, y seran guardados en un dataframe con nombre "data".

In [88]:
#importamos las librerias necesarias para el proyecto
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [89]:
pd.set_option('display.max_columns',None)
global data
data = pd.read_csv('/content/Drive/MyDrive/Tripleten projects/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

[Ahora vamos a explorar nuestros datos. Querrás ver cuántas columnas y filas hay, observa algunas filas para identificar posibles problemas con los datos.]

In [90]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
data.shape

(21525, 12)

In [91]:
#mostramos las primeras 10 filas
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


La columna days_employed muestra números en negativos, y se observa un dato excesivamente grande.

In [92]:
# Obtener información sobre los datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Existen valores nulos tanto en days_employed como en total_income, podria tratarse de un error en la entrada de datos.

In [93]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [94]:
# Veamos la cantidad de valores nulo por columna.
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Con el resultado e la expresión anterior, se puede apreciar que la cantidad de valores en ambas columnas es el mismo, por lo que suponemos que en cada registro que la columna days_employed tiene valor nulo, tambien lo tiene la columna total_income.

In [95]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
datos_filtrados=data[(data['days_employed'].isna()) & (data['total_income'].isna())]
datos_filtrados

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusión intermedia**

El numero de filas del filtro con ambas condiciones es el mismo que valores ausentes, esto significa que existe una relaccion directa en los valores asuentes de ambas columnas.

In [96]:
#Vamos a investigar la cantidad de valores nulos con respecto al total.
cantidad_na=len(datos_filtrados)
porcentaje_na=len(datos_filtrados)/len(data)*100
print('El porcentaje de valores ausentes es: {:.2f}%'.format(porcentaje_na))

El porcentaje de valores ausentes es: 10.10%


Vamos a investigar los valores nulos con respecto a las otras columnas, para buscar alguna relación o patrón.

In [97]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
# Se revisa la cantidad de valores nulos con respecto a la columna education
datos_filtrados['days_employed'].isna().groupby(datos_filtrados['education'],sort=False).count().sort_values(ascending=False)

education
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: days_employed, dtype: int64

In [98]:
# Se revisa la cantidad de valores nulos con respecto a la columna family_status
datos_filtrados['days_employed'].isna().groupby(datos_filtrados['family_status']).count().sort_values(ascending=False)

family_status
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: days_employed, dtype: int64

In [99]:
# Se revisa la cantidad de valores nulos con respecto a la columna income_type
datos_filtrados['days_employed'].isna().groupby(datos_filtrados['income_type']).count().sort_values(ascending=False)

income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: days_employed, dtype: int64

In [100]:
# Comprobando la distribución en el conjunto de datos entero
#Veamos los datos con respecto a income_type
data.groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

La mayor parte de valores ausentes correponden a empleados.

**Posibles razones por las que hay valores ausentes en los datos**

Al tener valores ausentes en esa columna exclusivamente se podria tratar de un error humano a la hora de introducir los datos.

**Conclusión intermedia**
Al igual que en los datos filtrados la mayor parte de datos corresponden a empleados, esto es porque globalmente la mayor cantidad de clientes son empleados. Lo cual no tiene ninguna relación directa que determine la razon de la falta de estos valores.
El error podria ser humano a la hora de introducir los datos, o un error del sistema al momento de la importación.

**Conclusiones**

No se encontró un patrón de pueda ayudar a determinar la razón en la ausencia de los valores, para poder rellenar los valores ausentes se utilizar las medias o medianas segun convenga, en relación con alguno de las otras columnas.

## Transformación de datos

Se revisa la columna education con el fin de ver los datos unicos, y examinar si es necesario relizar algún ajuste.

In [101]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
data.sort_values(by='education')['education'].unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

In [102]:
# Arregla los registros si es necesario
data['education']=data['education'].str.lower()

In [103]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
data.sort_values(by='education')['education'].unique()

array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

Se verifican los datos para la columna children

In [104]:
# Veamos la distribución de los valores en la columna `children`
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Existen datos negativos, y datos muy altos para el dato representan. Estos posiblemente se deben a errores humanos al momento de la captura de los datos.
Por lo que es necesario corregirlos, los datos en negativo (-1) se trasformarán a positivos y los datos muy grandes (20), se quitará el cero para dejarlos en unidades.


In [105]:
#suma de los datos atipicos en la columna children
child_atyp= data[data['children']<0]['children'].count()+ data[data['children']==20]['children'].count()
#porcentaje de datos atipicos en la columna children
child_atyp_per=child_atyp/data.shape[0]*100
print('El porcentaje de valores atipicos para la columna de Children es:{:.2f}%'.format(child_atyp_per))


El porcentaje de valores atipicos para la columna de Children es:0.57%


In [106]:
#registros con valor -1 se sustituye por 1
data.loc[(data['children']==-1),'children']=1
#registros con valor 20 se sutituye por 2
data.loc[(data['children']==20),'children']=2

Para arreglar los valores negativos, fueron sustituidos por el absoluto positivo, y para el caso de los valores con 20, se quito el cero, considerandolo un error a la hora de introducir los datos.

In [107]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Se verifican los datos en days_employed con el objeto de revisar si es necesaria alguna modificación.

In [108]:
data['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

Se detectaron valores con valores en negativo, lo cual no es posible para el dato que representan, y se detectan valores exageradamente grandes.

In [109]:
#seleccionamos como umbral para datos grandes los 20000 dias
#se calculan los dias trabajados negativos
dias_trab_neg=data[data['days_employed']<0]['days_employed'].count()
dias_trab_atyp=data[data['days_employed']>20000]['days_employed'].count()
print('Los registros con dias trabajados negativos son:',dias_trab_neg)
print('Los registros con dias trabajados con valores muy grandes son:',dias_trab_atyp)

Los registros con dias trabajados negativos son: 15906
Los registros con dias trabajados con valores muy grandes son: 3445


Los datos negativos posiblemente se deben a un error en el ingreso de los datos, por lo cual seran transformados a su equivalente en positivo para arreglar el error.
En el caso de los datos grandes al no encontrar una manera de determinar cuales son los valores correctos y al no ser un dato relevante para el objeto de este análisis, se opta por dejarlos tal cual.

In [110]:
# corregimos valores negativos
data.loc[data['days_employed']<0,'days_employed']=data['days_employed'].abs()

In [111]:
# Comprobamos el resultado
data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [112]:
# Revisamos `dob_years` en busca de valores sospechosos y contamos el porcentaje
data.sort_values(by='dob_years',ascending=False)['dob_years'].unique()

array([75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59,
       58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42,
       41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25,
       24, 23, 22, 21, 20, 19,  0])

Se detectan valores de cero, lo cual no es posible para el dato que representa esta columna, por lo cual se procede a revisar los registros con esta condición.

In [113]:
# Revisamos los problemas en la columna `dob_years`
data[data['dob_years']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


In [114]:
# Comprobamos cantidad y porcentaje de registros en esa condición
edades_en_cero=data[data['dob_years']==0]['dob_years'].count()
print('La cantidad de registros con edades en cero es:',edades_en_cero)
print('El porcentaje registros con edades en cero es:',edades_en_cero/data.shape[0]*100,'%')


La cantidad de registros con edades en cero es: 101
El porcentaje registros con edades en cero es: 0.4692218350754936 %


La cantidad de errores es baja en comparacion al total de registros y al no detectar un patrón que pueda ayudar a determinar el valor mas apropiado para asignarle, se opta por dejarlo tal cual, de igual forma dicho dato no es relevante para el objeto de este análisis.

In [115]:
# Veamos los valores de la columna family_status
data['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

No de detectan datos problemáticos en la columna family_status.

In [116]:
# Veamos los valores en la columna gender
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [117]:
#Veamos la cantidad de registros con XNA en la columna gender
data[data['gender']=='XNA']['gender'].count()

1

Solamente se detecta un resgistro con el valor de XNA en gender, se podría tratar de una persona que prefiere no especificar su género y al tratarse de un porcentaje muy pequeño se puede ya sea eliminar, despreciar o cambiar el valor por otro mas fácil de analizar, en este caso se optará por cambiar el valor por "Not specified".

In [118]:
#Los registros con el valor 'XNA' en gender se cambian por 'Not specified'
data.loc[data['gender']=='XNA','gender']='Not specified'
data['gender'].unique()

array(['F', 'M', 'Not specified'], dtype=object)

In [119]:
#Se comprueba la cantidad de registros por cada género.
data.groupby('gender')['gender'].count()

gender
F                14236
M                 7288
Not specified        1
Name: gender, dtype: int64

In [120]:
# Veamos los valores en la columna income_type
data['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

No se detectaron valores problemáticos en la columna income_type.

Ahora se verificara si existen valores duplicados en los datos.

In [121]:
# Comprobar los duplicados
data.duplicated().sum()


71

In [122]:
# Eliminamos duplicados
data=data.drop_duplicates().reset_index(drop=True)

In [123]:
# Última comprobación para ver si tenemos duplicados
data.duplicated().sum()

0

In [124]:
#Se revisan clientes retirados con edades menores de 40 años
data[(data['income_type']=='retiree') & (data['dob_years']<40)].sort_values(by='days_employed')['days_employed'].count()

53

Se detectarón 53 registros de personas que dicen ser retiradas con una edad de menos de 40, son un porcentaje minimo con respecto al total de registros por lo que procedemos a eliminarlos

In [125]:
#eliminamos los datos de personas retiradas menores de 40
data=data[~((data['income_type']=='retiree') & (data['dob_years']<40))]
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21450,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car




Se analizaron las columnas con datos ausentes y se investigaron las posbiles razones de estas omisiones, de igual manera de verificarón los datos problemáticos en las columnas, se encontraron valores negativos los cuales se transformaron a positivo, y se eliminaron valores duplicados, tanto en las columnas como en los registros completos.

# Trabajar con valores ausentes

### Restaurar valores ausentes en `total_income`

Existen valores ausentes tanto en la columna days_employed como en total_income, se revisarán primeramente los valores ausentes de total_income.

Con el objeto de categorizar de mejor manera los datos, se creara una columna con los valores de la edad por rangos.

In [126]:
# Vamos a escribir una función que calcule la categoría de edad
def cat_edad(edad):
    if edad<=20:
        return '0-20'
    if edad<=30:
        return '21-30'
    if edad <=40:
        return'31-40'
    if edad <=50:
        return '41-50'
    if edad <=60:
        return '51-60'
    return '60+'



In [127]:
# Creamos una nueva columna basada en la función
data['age_range']=data['dob_years'].apply(cat_edad)


In [128]:
#Verificamos la columna recién creada
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60


Con el fin de determinar los valores idóneo para poder completar los valores ausentes, se analizaran los factores de los que puede depender el ingreso de cada cliente, se analizará con respecto a education, family_status e income_type.

Se crea una tabla nueva sin valores ausentes, con el objeto de poder restaurar los valores ausentes de la tabla original.

In [129]:
# Se crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
datos_sin_na=data[~data['total_income'].isna()]
datos_sin_na

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41-50
21450,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60+
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,31-40
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,31-40


In [130]:
#Se crea una funcion para verificar las medias de total_income por cada uno de los factores
def cal_mean_fac(datos,columna,factor):
    try:
        medias=datos[columna].groupby(datos[factor]).mean()
        return medias
    except:
        return 'Ha ocurrido un error'

#Medias de total_income en base al factor education
print(cal_mean_fac(datos_sin_na,'total_income','education'))
print()
#Medias de total_income en base al factor family_status
print(cal_mean_fac(datos_sin_na,'total_income','family_status'))
print()
#Medias de total_income en base al factor income_type
print(cal_mean_fac(datos_sin_na,'total_income','income_type'))

education
bachelor's degree      33147.930790
graduate degree        27960.024667
primary education      21171.828427
secondary education    24609.368025
some college           29045.443644
Name: total_income, dtype: float64

family_status
civil partnership    26720.484676
divorced             27229.178758
married              27047.232571
unmarried            26953.486445
widow / widower      22993.290698
Name: total_income, dtype: float64

income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21943.745798
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64


In [131]:
# Se crea la funcion para verificar las medianas de total_income por cada uno de los factores
def cal_median_fac(datos,columna,factor):
    medianas=datos[columna].groupby(datos[factor]).median()
    return medianas
#Medianas de total_income en base al factor education
print(cal_median_fac(datos_sin_na,'total_income','education'))
print()
#Medianas de total_income en base al factor family_status
print(cal_median_fac(datos_sin_na,'total_income','family_status'))
print()
#Medianas de total_income en base al factor income_type
print(cal_median_fac(datos_sin_na,'total_income','income_type'))

education
bachelor's degree      28042.5420
graduate degree        25161.5835
primary education      18754.5480
secondary education    21854.6310
some college           25618.4640
Name: total_income, dtype: float64

family_status
civil partnership    23209.5485
divorced             23671.0585
married              23392.7690
unmarried            23161.0500
widow / widower      20518.7285
Name: total_income, dtype: float64

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        19057.1065
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64


Considerando las diferencias se utilizarán los valores con respecto a las columnas `age_range`, `income_type` y `education`, ya que por lo general estos son los factores que determinan los ingresos, de igual manera para que los sueldos muy grandes o muy bajos no afecten a la hora de determinar el valor idonéo para llenar los valores ausentes se utilizarán las medianas, tomando en cuenta el grado de estudios.


In [132]:
#  Escribimos la función que usaremos para completar los valores ausentes
def llenar_ausentes_mediana(tabla,columna,factor):
    try:
        #Por cada valor unico en el factor seleccionado
        for dato in tabla[factor].unique():
            #Se determina la mediana del factor unico por columna
            valor=tabla.loc[tabla[factor]==dato,columna].median()
            #Se llenan los valores ausentes de cada factor con la mediana calculada
            sustitucion=tabla[tabla[factor]==dato][columna].fillna(valor)
            #Se sustituyen los valores en la tabla original
            tabla.loc[tabla[factor]==dato,columna]=sustitucion

    except:
        return 'Ha ocurrido un error inesperado!!'

Con el objetivo de realizar el análisis tomando en cuenta varios factores, se creara una pivot table que contenga las medianas de la columna total_income, para poder utilizar estos valores y llenar los valores ausentes que se tienen.

In [133]:
#Se genera una pivot table para obtener las medianas de 'total_income' en base a 3 factores age_range,income_type y education
tabla_analisis=datos_sin_na.pivot_table(index=['age_range','income_type'],columns='education',
                                        values='total_income',aggfunc='median')
tabla_analisis

education                              bachelor's degree  graduate degree  \
age_range income_type                                                       
0-20      business                            34476.3210              NaN   
          civil servant                       18335.8510              NaN   
          employee                            21085.2470              NaN   
21-30     business                            28458.9030              NaN   
          civil servant                       24886.2320              NaN   
          employee                            24556.6440              NaN   
          entrepreneur                        79866.1030              NaN   
          student                             15712.2600              NaN   
31-40     business                            32995.1480              NaN   
          civil servant                       27956.2445        17822.757   
          employee                            26948.8590        18551.846   
          paternity / maternity leave                NaN              NaN   
          retiree                                    NaN              NaN   
          unemployed                                 NaN              NaN   
41-50     business                            34616.9110              NaN   
          civil servant                       28701.5505              NaN   
          employee                            27110.1080        31771.321   
          retiree                             32901.6320              NaN   
          unemployed                          32435.6020              NaN   
51-60     business                            36057.4025              NaN   
          civil servant                       27665.2370              NaN   
          employee                            27979.3760        42945.794   
          retiree                             22679.1140              NaN   
60+       business                            30144.1315              NaN   
          civil servant                       37432.4595              NaN   
          employee                            27989.3610              NaN   
          retiree                             23030.2470        28334.215   

education                              primary education  secondary education  \
age_range income_type                                                           
0-20      business                                   NaN           21517.7095   
          civil servant                              NaN           21845.9410   
          employee                                   NaN           19769.9910   
21-30     business                            21167.1430           23953.9550   
          civil servant                       30563.3830           20963.9380   
          employee                            23533.5420           20295.2120   
          entrepreneur                               NaN                  NaN   
          student                                    NaN                  NaN   
31-40     business                            21441.9210           26320.6530   
          civil servant                       21150.6960           21990.3075   
          employee                            19674.2825           22210.3235   
          paternity / maternity leave                NaN            8612.6610   
          retiree                                    NaN           13256.7850   
          unemployed                                 NaN            9593.1190   
41-50     business                            29564.1760           25905.9750   
          civil servant                       78410.7740           21825.8030   
          employee                            21204.0860           22108.7565   
          retiree                             16686.8840           22209.5085   
          unemployed                                 NaN                  NaN   
51-60     business                            23066.1310         

In [134]:
#Se define funcion que busque combinaciones de los 3 factores seleccionados y devuelva la mediana
def mediana_income(row):
    try:
        age=row['age_range']
        tipo=row['income_type']
        education=row['education']
        return tabla_analisis.loc[(age,tipo)][education]
    except:
        return float('NaN')
#Se define una fila para probar la función anteriormente definida
fila_test=pd.Series(data=['0-20','business',"some college",],index=['age_range','income_type','education',])
mediana_income(fila_test)

16020.785

In [135]:
#Se aplica la funcion anteriormente definida a la tabla principal (data)
data.loc[data['total_income'].isna(),'total_income']=data.apply(mediana_income,axis=1)

In [136]:
data.loc[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
3610,0,NaN,40,bachelor's degree,0,married,0,M,retiree,0,NaN,housing renovation,31-40
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,51-60
8133,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,NaN,to have a wedding,60+


Se encontraron 3 valores nulos en total_income, esto es debido a que no existe un valor en la tabla_analisis con esa combinación de factores, al ser solo un valor se tomara la combinación mas cercana con respecto a su rango de edades, es decir los rengos de 31-40 y 51-60.

In [137]:
#Se define una fila para llenar el valor ausente de 51-60
fila_prueba=pd.Series(data=['51-60','civil servant',"primary education",],index=['age_range','income_type','education',])
#Se aplica la funcion en la fila ausente
data.loc[data['total_income'].isna(),'total_income']=mediana_income(fila_prueba)

In [138]:
#Se verifican los resultados de aplicar las funcion a las filas
data.isna().sum()

children               0
days_employed       2094
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_range              0
dtype: int64

In [139]:
# Comprobar el número de entradas en las columnas
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21392 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21392 non-null  int64  
 1   days_employed     19298 non-null  float64
 2   dob_years         21392 non-null  int64  
 3   education         21392 non-null  object 
 4   education_id      21392 non-null  int64  
 5   family_status     21392 non-null  object 
 6   family_status_id  21392 non-null  int64  
 7   gender            21392 non-null  object 
 8   income_type       21392 non-null  object 
 9   debt              21392 non-null  int64  
 10  total_income      21392 non-null  float64
 11  purpose           21392 non-null  object 
 12  age_range         21392 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.8+ MB


###  Restaurar valores en `days_employed`

Para el caso de la columna days_employed se realizará un tratamiento similar al de la columna total_income, se utilizarán los mismos factores, age_range, income_type y education, de la misma manera se utilizara la mediana ya que desde inicio se detectó que existen valores atipicos en la columna y estos valores hacen que la media tienda hacia ellos lo que puede causar errores en el análisis.

In [140]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
# Se crea una pivot table para obtener la mediana de days_employed en base a los factores anteriormente definidos.
tabla_days=datos_sin_na.pivot_table(index=['age_range','income_type'],columns='education',
                                        values='days_employed',aggfunc='median')
tabla_days

education                              bachelor's degree  graduate degree  \
age_range income_type                                                       
0-20      business                            986.789230              NaN   
          civil servant                      4774.343347              NaN   
          employee                           1095.386622              NaN   
21-30     business                            983.266608              NaN   
          civil servant                      1497.928370              NaN   
          employee                           1071.716295              NaN   
          entrepreneur                        520.848083              NaN   
          student                             578.751554              NaN   
31-40     business                           1407.728133              NaN   
          civil servant                      2829.971842      5968.075884   
          employee                           1556.457266      2351.431934   
          paternity / maternity leave                NaN              NaN   
          retiree                                    NaN              NaN   
          unemployed                                 NaN              NaN   
41-50     business                           1931.178446              NaN   
          civil servant                      3866.851554              NaN   
          employee                           1953.775975       409.200149   
          retiree                          363926.256740              NaN   
          unemployed                       395302.838654              NaN   
51-60     business                           2016.517713              NaN   
          civil servant                      4152.558995              NaN   
          employee                           2588.545936      5352.038180   
          retiree                          364689.164541              NaN   
60+       business                           2576.120198              NaN   
          civil servant                      4448.167468              NaN   
          employee                           3000.270035              NaN   
          retiree                          367857.806545    356930.517546   

education                              primary education  secondary education  \
age_range income_type                                                           
0-20      business                                   NaN           674.838979   
          civil servant                              NaN          4847.111204   
          employee                                   NaN          1019.354207   
21-30     business                            856.198482          1024.652083   
          civil servant                       760.901517          1677.407868   
          employee                            976.611688          1064.560075   
          entrepreneur                               NaN                  NaN   
          student                                    NaN                  NaN   
31-40     business                           1347.644931          1641.266192   
          civil servant                      1672.633661          2675.464186   
          employee                           1235.971542          1593.376896   
          paternity / maternity leave                NaN          3296.759962   
          retiree                                    NaN        370595.945616   
          unemployed                                 NaN        337524.466835   
41-50     business                            995.775599          1915.961625   
          civil servant                      4193.718465          3229.254557   
          employee                            927.789271          2017.225673   
          retiree                          367538.444192        366764.969526   
          unemployed                                 NaN                  NaN   
51-60     business                           2916.911778         

In [141]:
#Se define funcion que busque combinaciones de los 3 factores y devuelva la mediana en este caso para la columna days_employed
def mediana_days(row):
    try:
        age=row['age_range']
        tipo=row['income_type']
        education=row['education']
        return tabla_days.loc[(age,tipo)][education]
    except:
        return float('NaN')
#Se define una fila para probar la función anteriormente definida
fila_test=pd.Series(data=['0-20','business',"some college",],index=['age_range','income_type','education',])
mediana_days(fila_test)

695.9689513241333

In [142]:
#Se aplica la funcion anteriormente definida a la tabla principal (data)
data.loc[data['days_employed'].isna(),'days_employed']=data.apply(mediana_days,axis=1)

In [143]:
#Se verifica el resultado de la función
data.isna().sum()

children            0
days_employed       3
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_range           0
dtype: int64

In [144]:
data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
3610,0,NaN,40,bachelor's degree,0,married,0,M,retiree,0,12709.2755,housing renovation,31-40
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,12709.2755,buy residential real estate,51-60
8133,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,12709.2755,to have a wedding,60+


Se encontraron 3 valores nulos en total_income, esto es debido a que no existe un valor en la tabla_days con esa combinación de factores, al ser solo un valor se tomara la combinación mas cercana con respecto a su rango de edades, es decir el rango de 51-60.

In [145]:
#Se define una fila para llenar el valor ausente
fila_ajuste=pd.Series(data=['51-60','civil servant',"primary education",],index=['age_range','income_type','education',])
#Se aplica la funcion en la fila ausente
data.loc[data['days_employed'].isna(),'days_employed']=mediana_days(fila_prueba)

In [146]:
#Se verifican los resultados.
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_range           0
dtype: int64

## Clasificación de datos

Con el objeto de clasificar los valores y poder contestar las preguntas de las hipótesis, se verificarán los valores unicos de las columnas children, family_status, total_income y purpose, para saber si es necesario realizar una clasificación en los datos


In [148]:
#Se verifican los valores únicos de la columna children
data.sort_values(by='children',ascending=False)['children'].unique()

array([5, 4, 3, 2, 1, 0])

Con el resultado de los valores unicos de la columna children, se concluye que no es necesario reclasificar los datos ya que son pocos los valores únicos.

In [149]:
# Comprobar los valores únicos de la columna family_status
data.sort_values(by='family_status',ascending=True)['family_status'].unique()

array(['civil partnership', 'divorced', 'married', 'unmarried',
       'widow / widower'], dtype=object)

De igual forma que con la columna anterior son pocos valores únicos para reclasificar.

Se verifican los datos unicos de la columna total_income con el objeto de poder determinar una clasificación.

In [158]:
#Revisar los valor unicos de total_income
data.sort_values(by='total_income',ascending=False)['total_income'].unique()

array([362496.645, 352136.354, 276204.162, ...,   3471.216,   3418.824,
         3306.762])

In [160]:
#Contamos la cantidad de valores únicos de la columna total_income
data.sort_values(by='total_income',ascending=False)['total_income'].unique().size

19327

In [161]:
# Obtener estadísticas resumidas para la columna
data['total_income'].describe()

count     21392.000000
mean      26490.252773
std       15739.269993
min        3306.762000
25%       17223.821250
50%       22939.742000
75%       31656.845250
max      362496.645000
Name: total_income, dtype: float64

Con el objeto se poder visualizar mejor los datos se crearan 4 rangos para clasificar los valores de total_income, 0-50k, 50k-100k, 100k-150k y 150k-200k y 200k+.

In [162]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
#Se utiliza pd.cut
rangos=[0,50000,100000,150000,200000,np.inf]
nombres_grupos=['0-50K','50K-100K','100K-150K','150K-200K','200K+']

# Crear una columna con categorías
data['income_range']=pd.cut(data['total_income'],bins=rangos,labels=nombres_grupos)

In [163]:
# Comprobamos los resultados
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range,income_range
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50,0-50K
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40,0-50K
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40,0-50K
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40,0-50K
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60,0-50K
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,21-30,0-50K
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41-50,0-50K
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50,0-50K
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,31-40,0-50K
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41-50,0-50K


Es necesario revisar los propósitos de los prestamos para poder clasificar los temas comunes.

In [164]:
# Observamos los valores únicos de la columna purpose
data.sort_values(by='purpose',ascending=False)['purpose'].unique()

array(['wedding ceremony', 'university education',
       'transactions with my real estate',
       'transactions with commercial real estate', 'to own a car',
       'to have a wedding', 'to get a supplementary education',
       'to buy a car', 'to become educated', 'supplementary education',
       'second-hand car purchase', 'real estate transactions',
       'purchase of the house for my family', 'purchase of the house',
       'purchase of my own house', 'purchase of a car', 'property',
       'profile education', 'housing transactions', 'housing renovation',
       'housing', 'having a wedding', 'going to university',
       'getting higher education', 'getting an education', 'education',
       'construction of own property', 'cars', 'car purchase', 'car',
       'buying property for renting out', 'buying my own car',
       'buying a second-hand car', 'buy residential real estate',
       'buy real estate', 'buy commercial real estate',
       'building a real estate', 'build

In [165]:
# Se define una función para clasificar los propositos de los clientes
def clas_purpose(texto):
    try:
        if 'wed'in texto:
            return 'Wedding'
        if 'car'in texto:
            return 'Get a car'
        if ('hous'in texto or 'state'in texto or 'property'in texto):
            return 'Get a house/property'
        if('educ'in texto or 'university'in texto):
            return 'Education'
        else:
            return 'Other'
    except:
        return 'Ha ocurrido un error'


In [166]:
#Verificar el funcionamiento de la función
print(clas_purpose('wedding ceremony'))
print(clas_purpose('real estate transactions'))
print(clas_purpose('buying a second-hand car'))
print(clas_purpose('to become educated'))

Wedding
Get a house/property
Get a car
Education


In [167]:
#Se utiliza la funcion para crear la nueva columna
data['clasified_purpose']=data['purpose'].apply(clas_purpose)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range,income_range,clasified_purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50,0-50K,Get a house/property
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40,0-50K,Get a car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40,0-50K,Get a house/property
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40,0-50K,Education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60,0-50K,Wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,21-30,0-50K,Get a house/property
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41-50,0-50K,Get a house/property
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50,0-50K,Education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,31-40,0-50K,Wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41-50,0-50K,Get a house/property


## Comprobación de las hipótesis

**¿Existe una correlación entre tener hijos y pagar a tiempo?**

Para contestar esta pregunta es necesario clasificar los registros con adeudos en base a la cantidad de hijos que tiene cada cliente.

In [168]:
# Se crea una pivot table para verificar la tasa de incumplimiento
tabla_children=data.pivot_table(index='children',columns='debt',values='dob_years',aggfunc='count',
                               fill_value=0)
tabla_children

debt,0,1
children,,
0,12982,1060
1,4405,444
2,1922,200
3,302,27
4,37,4
5,9,0


In [169]:
#Se calcula la tasa de inclumplimiento en base a la cantidad de hijos del cliente
tabla_children['tasa_incumplimiento']=round(tabla_children.loc[:,1]/(tabla_children.loc[:,1]+tabla_children.loc[:,0])*100,2)
tabla_children

debt,0,1,tasa_incumplimiento
children,,,
0,12982,1060,7.55
1,4405,444,9.16
2,1922,200,9.43
3,302,27,8.21
4,37,4,9.76
5,9,0,0.00


**Conclusión**

En base al resultado de la tabla anterior que concluye que la tasa de endeudamiento es menor en los clientes que no tienen hijos, los clientes que tienen entre 1 y 4 hijos suelen tener aproximadamente un 9% de probabilidad de que se endeuden.



**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [170]:
#Creamos una pivot table para relacionar ambos factores
tabla_family=data.pivot_table(index='family_status',columns='debt',values='dob_years',aggfunc='count')
tabla_family

debt,0,1
family_status,,
civil partnership,3750,387
divorced,1103,85
married,11388,928
unmarried,2525,274
widow / widower,891,61


In [171]:
#obtenemos la tasa de incumplimiento en base a la situación familiar
tabla_family['tasa_incumplimiento']=round(tabla_family.loc[:,1]/(tabla_family.loc[:,1]+tabla_family.loc[:,0])*100,2)
tabla_family

debt,0,1,tasa_incumplimiento
family_status,,,
civil partnership,3750,387,9.35
divorced,1103,85,7.15
married,11388,928,7.53
unmarried,2525,274,9.79
widow / widower,891,61,6.41


**Conclusión**

Los clientes divorciados, casados y viudos tienen una tasa de endeudamiento menor a los que son solteros o viven en unión civil.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [172]:
tabla_inc_ran=data.pivot_table(index='income_range',columns='debt',values='dob_years',aggfunc='count')
tabla_inc_ran

debt,0,1
income_range,,
0-50K,18431,1643
50K-100K,1133,86
100K-150K,67,4
150K-200K,16,1
200K+,10,1


In [173]:
tabla_inc_ran['tasa_incumplimiento']=round(tabla_inc_ran.loc[:,1]/(tabla_inc_ran.loc[:,1]+tabla_inc_ran.loc[:,0])*100,2)
tabla_inc_ran

debt,0,1,tasa_incumplimiento
income_range,,,
0-50K,18431,1643,8.18
50K-100K,1133,86,7.05
100K-150K,67,4,5.63
150K-200K,16,1,5.88
200K+,10,1,9.09


**Conclusión**

En base a la tabla anterior se puede concluir que a mayor ingreso la tasa de endeudamiento es mas baja, se onbserva un caso curioso para los clientes con el mayor ingreso, pero al ser una cantidad pequeña no es un resultado muy confiable.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**


Buscamos la relación entre el propósito del prestamo y el pago a tiempo

In [174]:
tabla_purpose=data.pivot_table(index='clasified_purpose',columns='debt',values='dob_years',aggfunc='count')
tabla_purpose

debt,0,1
clasified_purpose,,
Education,3629,369
Get a car,3895,399
Get a house/property,10004,782
Wedding,2129,185


In [175]:
tabla_purpose['tasa_incumplimiento']=round(tabla_purpose.loc[:,1]/(tabla_purpose.loc[:,1]+tabla_purpose.loc[:,0])*100,2)
tabla_purpose

debt,0,1,tasa_incumplimiento
clasified_purpose,,,
Education,3629,369,9.23
Get a car,3895,399,9.29
Get a house/property,10004,782,7.25
Wedding,2129,185,7.99


**Conclusión**

Cuando el crédito se solicita para la adquisición o renta de alguna propiedad, ya sea comercial o residencial, la tasa de endeudamiento es mas baja, para proposito de bodas, sube un poco y para casos de educación o conseguir un vehículo se incrementa la tasa.


# Conclusión general #

Antes de poder realizar un análisis a los datos fue necesario primero examinar y transformar los datos despues de la importación, primero se revisaron valores similares en datos de texto, con el objeto de clasificar de mejor manera, se revisaron que los valores de columnas numéricas tuvieran coherencia con el dato que representaban, ya que se encontraron valores negativos los cuales son irreales.

Para el caso de los valores ausentes se utilizaron las medianas esto debido a que existian valores atipicos los cuales generaban datos irreales.

Con el análisis anterior se respondieron las siguientes preguntas:

1.-¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?.
Se concluye que la tasa de endeudamiento es menor en los clientes que no tienen hijos, los clientes que tienen entre 1 y 4 hijos suelen tener aproximadamente un 9% de probabilidad de que se endeuden, posiblemente se debe a que los clientes sin hijos podrian tener como prioridad alta el pago de sus deudas.

2.-¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?.

Los clientes divorciados, casados y viudos tienen una tasa de endeudamiento menor a los que son solteros o viven en unión civil, esto podria deberse a las responsabilidades o prioridades de los clientes, por lo regular los que han estado casados tienen un poco mas de responsabilidad para pagar las deudas.


3.-¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?.
El ingreso tiene relacion directa con la tasa de endeudamiento, a mayor ingreso es menos probable que el cliente se endeude.

4.-¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?.

Cuando el crédito se solicita para la adquisición o renta de alguna propiedad, ya sea comercial o residencial, la tasa de endeudamiento es mas baja, para proposito de bodas, sube un poco y para casos de educación o conseguir un vehículo se incrementa la tasa.

Para un futuro, se podria utilizar el monto del crédito solicitado, asi como el plazo del mismo, ya que estos en combinación con los otros factores podrian ayudar a determinar de una maneja mas eficiente las tasas de endeudamiento.